# Final Experiments

### Plotting

In [78]:
import matplotlib.pyplot as plt
from math import floor
from matplotlib import gridspec
from lib.clustering_helpers import *

"""
Problem-specific plotting
"""
class Plot:
    def __init__(self, x=10, y=10):
        plt.rcParams["figure.autolayout"] = True
        self.fig = plt.figure(figsize=(y,x))
        self.cnt = 0
        self.new_ax = None

    def add(self, y, label=None):
        if label != None:
            self.new_ax.plot(y, label=label)
            self.new_ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),
                                 ncol = 2, fancybox=True, shadow=True,
                                 handleheight=2.4, labelspacing=0.05)
        else:
            self.new_ax.plot(y)

    def inc(self, ylabel="", xlabel=""):
        """Plots the data to a new subplot at the bottom."""
        self.cnt += 1
        gs = gridspec.GridSpec(self.cnt, 1)

        # Reposition existing subplots
        for i, ax in enumerate(self.fig.axes):
            ax.set_position(gs[i].get_position(self.fig))
            ax.set_subplotspec(gs[i])

        # Add new subplot
        self.new_ax = self.fig.add_subplot(gs[self.cnt-1])
        self.new_ax.set_xlabel(xlabel, fontsize=16)
        self.new_ax.set_ylabel(ylabel, fontsize=16)
        
        
    def add_new(self, y, label="No Label"):
        self.inc()
        self.add(y, label)

    def add_new_assignment_plot(self, assignments, labels=[], fuzzy=False, length=0, smoothness=1, xlabel="", ylabel=""):
        self.inc(ylabel, xlabel)
        for idx, ts in enumerate(assignments):
            if not fuzzy:
                y = []
                if length == 0:
                    length = max( [ max(ts) for ts in assignments if ts != [] ] )
                for t in range(length):
                    if t in ts:
                        y.append(1)
                    else:
                        y.append(0)
            else:
                y = ts
            if max(y) > 0:
                if labels == None:
                    self.add(smoothed(y,smoothness))
                elif labels == []:
                    self.add(smoothed(y, smoothness), idx)
                else:
                    self.add(smoothed(y, smoothness), str(labels[idx]))


    def add_new_conceptors_fit_plot(self, X, Cs, Ns=None, label="", labels="", smoothness=3):
        """
        Plots, for each time step t, how well each conceptor in Cs matches the state x(t)
        """
        self.inc()
        if Ns:
            collection = evidences_for_Cs(X,Cs,Ns)
        else:
            collection, _ = test(X, Cs, "PROP")
        if labels:
            for vals, label in zip(collection, labels):
                # walking average of d
                self.add(smoothed(vals, smoothness), label=label)
        else:                
            for i, vals in enumerate(collection):
                # walking average of d
                self.add(smoothed(vals, smoothness), label=label+str(i))


    def finalize(self, title=""):
        self.fig.suptitle(title, fontsize=16)
        plt.show()

### Collect training features

In [92]:
import os, sys
import importlib
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('/Users/joris/Documents/Work/thesis/Bachelor-Thesis/Code/hierarchical-clustering/Final Classification.ipynb'))))
import matplotlib.pyplot as plt
import pickle as pkl
import gc
import numpy as np
np.random.seed(0)
import random
random.seed(0)
from dataset.loading import Feature_Collector
from lib.conceptors import *
from lib.esn import ESN
from lib.helpers import *
import pandas as pd
import warnings
import json
warnings.filterwarnings("ignore")


path = '../timit/'
fc = Feature_Collector(path)

save = True

dr = []
fc.readTrainingDataDescriptionCSV()
speakers = fc._Tdd.speaker_id.unique()
df = pd.DataFrame({"speaker_id":speakers})
df['gender'] = df.speaker_id.str[0]
speakers = df.groupby("gender").sample(50, replace=False).speaker_id.values
sentence = "SX"

long_version = False
n_mels = 13
delta = False
delta_delta = False
subsamples = 10

path_option = "Final"+str(long_version)+str(n_mels)+str(delta)+str(delta_delta)+str(subsamples)

if dr:
    path_option = str(dr)+"_"+path_option
if len(speakers):
    path_option = str(speakers[0])+"_"+path_option
    
features,labels,oversamplings = fc.collectFeaturesInSegments(
    n_mels=n_mels,delta=delta,delta_delta=delta_delta,
    long_version=long_version,speakers=speakers,dr=dr,
    sentence=sentence,subsamples=subsamples,path_option=path_option)

gc.collect()

-from output
---- success


0

### Regroup data and subset phonemes

In [93]:
def filter_data(features, labels, classes=None,trunkate=False,limit=1000000):
    group = {}

    for i in range(len(labels)):
        if classes != None and labels[i] not in classes:
            continue
        if labels[i] not in group.keys():
            group[labels[i]] = []
        if len(group[labels[i]]) < limit:
            group[labels[i]].append(features[i])

    samples_per_phoneme = min([ len(value) for value in group.values() ])
    if trunkate:
        for key in group.keys():
            group[key] = random.sample(group[key], samples_per_phoneme)

    classes = list(group.keys())
    filtered_labels = []
    filtered_features = []
    for label, features in group.items():
        for feature in features:
            filtered_features.append(feature)
            filtered_labels.append(label)
    return classes, filtered_features, group, filtered_labels

group = {}
#selected_labels = None
#selected_labels = ["aa", "b", "s", "iy"]
selected_labels = ["aa", "ae", "ah", "eh", "ih", "iy", "uh"]
#selected_labels = ["aa", "ae", "ah", "eh", "ih", "iy", "uh", "er", "ey", "ix", "aw", "axr", "l", "oy", "r", "y"]

classes, features, group, labels = filter_data(features, labels, classes=selected_labels,trunkate=False,limit=1000000)

print(len(features))

n_samples = len(features)
phonemes = list(group.keys())
#samples_per_phoneme = min([ len(value) for value in group.values() ])

idx=0
correct_assignments = [ [] for _ in group.values() ]
for i,vals in enumerate(group.values()):
    for val in vals:
        correct_assignments[i].append(idx)
        idx += 1
        
print(str(len(phonemes))+" phonemes")
print("Number of samples:", n_samples)
print(phonemes)

4223
7 phonemes
Number of samples: 4223
['ih', 'ae', 'ah', 'iy', 'aa', 'eh', 'uh']


### Initialize Reservoir

In [94]:
aperture = 1

XorZ = "X"
method = "ogsignals"
#method = "centroids"
#method = "sims"
#method = "pred"

esn_params = {
    "in_dim": n_mels*(1+delta+delta_delta),
    "out_dim": n_mels*(1+delta+delta_delta),
    "N": (XorZ=="X")*80+20,
    "W_in_scale": 1.1,
    "b_scale": .44,
    "spectral_radius": 2.57,
    "weights": .1

}

esn = ESN(esn_params)

## Kmeans (Below-phoneme)
### Compute conceptors

In [95]:
save = False
cache_dir = '../cache/'
opt = True
normalize = True

file_name = "Below_phoneme"+str(opt)+XorZ+str(aperture)+str(esn_params["N"])+str(esn_params["W_in_scale"])+str(esn_params["b_scale"])+str(esn_params["spectral_radius"])+path_option+str(selected_labels)
    
if save and os.path.exists(cache_dir+"Cs"+file_name+'.pkl'):
    print("- loading conceptors from file")
    Cs_fp = open(cache_dir+"Cs"+file_name+'.pkl','rb')
    Cs = pkl.load(Cs_fp)
    Cs_fp.close()
    print("--- Done")
else:
    print("- computing conceptors")
    Cs = []
    for phoneme, signals in group.items():
        for signal in signals:
            x = esn.run(signal.T, XorZ=XorZ)
            Cs.append(compute_c(x, aperture))
    if opt:
        print("- optimizing +")
        Cs = optimize_apertures(Cs)
    if normalize:
        Cs, opt_singular_val_sum = normalize_apertures(Cs)
    
    if save:
        Cs_fp = open(cache_dir+"Cs"+file_name+".pkl",'wb')
        pkl.dump(Cs,Cs_fp)
        Cs_fp.close()
    
    print("--- Done")

- computing conceptors
- optimizing +
Computing gammas...
1  of  4223
2  of  4223
3  of  4223
4  of  4223
5  of  4223
6  of  4223
7  of  4223
8  of  4223
9  of  4223
10  of  4223
11  of  4223
12  of  4223
13  of  4223
14  of  4223
15  of  4223
16  of  4223
17  of  4223
18  of  4223
19  of  4223
20  of  4223
21  of  4223
22  of  4223
23  of  4223
24  of  4223
25  of  4223
26  of  4223
27  of  4223
28  of  4223
29  of  4223
30  of  4223
31  of  4223
32  of  4223
33  of  4223
34  of  4223
35  of  4223
36  of  4223
37  of  4223
38  of  4223
39  of  4223
40  of  4223
41  of  4223
42  of  4223
43  of  4223
44  of  4223
45  of  4223
46  of  4223
47  of  4223
48  of  4223
49  of  4223
50  of  4223
51  of  4223
52  of  4223
53  of  4223
54  of  4223
55  of  4223
56  of  4223
57  of  4223
58  of  4223
59  of  4223
60  of  4223
61  of  4223
62  of  4223
63  of  4223
64  of  4223
65  of  4223
66  of  4223
67  of  4223
68  of  4223
69  of  4223
70  of  4223
71  of  4223
72  of  4223
73  of  4223
74

590  of  4223
591  of  4223
592  of  4223
593  of  4223
594  of  4223
595  of  4223
596  of  4223
597  of  4223
598  of  4223
599  of  4223
600  of  4223
601  of  4223
602  of  4223
603  of  4223
604  of  4223
605  of  4223
606  of  4223
607  of  4223
608  of  4223
609  of  4223
610  of  4223
611  of  4223
612  of  4223
613  of  4223
614  of  4223
615  of  4223
616  of  4223
617  of  4223
618  of  4223
619  of  4223
620  of  4223
621  of  4223
622  of  4223
623  of  4223
624  of  4223
625  of  4223
626  of  4223
627  of  4223
628  of  4223
629  of  4223
630  of  4223
631  of  4223
632  of  4223
633  of  4223
634  of  4223
635  of  4223
636  of  4223
637  of  4223
638  of  4223
639  of  4223
640  of  4223
641  of  4223
642  of  4223
643  of  4223
644  of  4223
645  of  4223
646  of  4223
647  of  4223
648  of  4223
649  of  4223
650  of  4223
651  of  4223
652  of  4223
653  of  4223
654  of  4223
655  of  4223
656  of  4223
657  of  4223
658  of  4223
659  of  4223
660  of  4223
661  o

1164  of  4223
1165  of  4223
1166  of  4223
1167  of  4223
1168  of  4223
1169  of  4223
1170  of  4223
1171  of  4223
1172  of  4223
1173  of  4223
1174  of  4223
1175  of  4223
1176  of  4223
1177  of  4223
1178  of  4223
1179  of  4223
1180  of  4223
1181  of  4223
1182  of  4223
1183  of  4223
1184  of  4223
1185  of  4223
1186  of  4223
1187  of  4223
1188  of  4223
1189  of  4223
1190  of  4223
1191  of  4223
1192  of  4223
1193  of  4223
1194  of  4223
1195  of  4223
1196  of  4223
1197  of  4223
1198  of  4223
1199  of  4223
1200  of  4223
1201  of  4223
1202  of  4223
1203  of  4223
1204  of  4223
1205  of  4223
1206  of  4223
1207  of  4223
1208  of  4223
1209  of  4223
1210  of  4223
1211  of  4223
1212  of  4223
1213  of  4223
1214  of  4223
1215  of  4223
1216  of  4223
1217  of  4223
1218  of  4223
1219  of  4223
1220  of  4223
1221  of  4223
1222  of  4223
1223  of  4223
1224  of  4223
1225  of  4223
1226  of  4223
1227  of  4223
1228  of  4223
1229  of  4223
1230  of  

1711  of  4223
1712  of  4223
1713  of  4223
1714  of  4223
1715  of  4223
1716  of  4223
1717  of  4223
1718  of  4223
1719  of  4223
1720  of  4223
1721  of  4223
1722  of  4223
1723  of  4223
1724  of  4223
1725  of  4223
1726  of  4223
1727  of  4223
1728  of  4223
1729  of  4223
1730  of  4223
1731  of  4223
1732  of  4223
1733  of  4223
1734  of  4223
1735  of  4223
1736  of  4223
1737  of  4223
1738  of  4223
1739  of  4223
1740  of  4223
1741  of  4223
1742  of  4223
1743  of  4223
1744  of  4223
1745  of  4223
1746  of  4223
1747  of  4223
1748  of  4223
1749  of  4223
1750  of  4223
1751  of  4223
1752  of  4223
1753  of  4223
1754  of  4223
1755  of  4223
1756  of  4223
1757  of  4223
1758  of  4223
1759  of  4223
1760  of  4223
1761  of  4223
1762  of  4223
1763  of  4223
1764  of  4223
1765  of  4223
1766  of  4223
1767  of  4223
1768  of  4223
1769  of  4223
1770  of  4223
1771  of  4223
1772  of  4223
1773  of  4223
1774  of  4223
1775  of  4223
1776  of  4223
1777  of  

2258  of  4223
2259  of  4223
2260  of  4223
2261  of  4223
2262  of  4223
2263  of  4223
2264  of  4223
2265  of  4223
2266  of  4223
2267  of  4223
2268  of  4223
2269  of  4223
2270  of  4223
2271  of  4223
2272  of  4223
2273  of  4223
2274  of  4223
2275  of  4223
2276  of  4223
2277  of  4223
2278  of  4223
2279  of  4223
2280  of  4223
2281  of  4223
2282  of  4223
2283  of  4223
2284  of  4223
2285  of  4223
2286  of  4223
2287  of  4223
2288  of  4223
2289  of  4223
2290  of  4223
2291  of  4223
2292  of  4223
2293  of  4223
2294  of  4223
2295  of  4223
2296  of  4223
2297  of  4223
2298  of  4223
2299  of  4223
2300  of  4223
2301  of  4223
2302  of  4223
2303  of  4223
2304  of  4223
2305  of  4223
2306  of  4223
2307  of  4223
2308  of  4223
2309  of  4223
2310  of  4223
2311  of  4223
2312  of  4223
2313  of  4223
2314  of  4223
2315  of  4223
2316  of  4223
2317  of  4223
2318  of  4223
2319  of  4223
2320  of  4223
2321  of  4223
2322  of  4223
2323  of  4223
2324  of  

2805  of  4223
2806  of  4223
2807  of  4223
2808  of  4223
2809  of  4223
2810  of  4223
2811  of  4223
2812  of  4223
2813  of  4223
2814  of  4223
2815  of  4223
2816  of  4223
2817  of  4223
2818  of  4223
2819  of  4223
2820  of  4223
2821  of  4223
2822  of  4223
2823  of  4223
2824  of  4223
2825  of  4223
2826  of  4223
2827  of  4223
2828  of  4223
2829  of  4223
2830  of  4223
2831  of  4223
2832  of  4223
2833  of  4223
2834  of  4223
2835  of  4223
2836  of  4223
2837  of  4223
2838  of  4223
2839  of  4223
2840  of  4223
2841  of  4223
2842  of  4223
2843  of  4223
2844  of  4223
2845  of  4223
2846  of  4223
2847  of  4223
2848  of  4223
2849  of  4223
2850  of  4223
2851  of  4223
2852  of  4223
2853  of  4223
2854  of  4223
2855  of  4223
2856  of  4223
2857  of  4223
2858  of  4223
2859  of  4223
2860  of  4223
2861  of  4223
2862  of  4223
2863  of  4223
2864  of  4223
2865  of  4223
2866  of  4223
2867  of  4223
2868  of  4223
2869  of  4223
2870  of  4223
2871  of  

3352  of  4223
3353  of  4223
3354  of  4223
3355  of  4223
3356  of  4223
3357  of  4223
3358  of  4223
3359  of  4223
3360  of  4223
3361  of  4223
3362  of  4223
3363  of  4223
3364  of  4223
3365  of  4223
3366  of  4223
3367  of  4223
3368  of  4223
3369  of  4223
3370  of  4223
3371  of  4223
3372  of  4223
3373  of  4223
3374  of  4223
3375  of  4223
3376  of  4223
3377  of  4223
3378  of  4223
3379  of  4223
3380  of  4223
3381  of  4223
3382  of  4223
3383  of  4223
3384  of  4223
3385  of  4223
3386  of  4223
3387  of  4223
3388  of  4223
3389  of  4223
3390  of  4223
3391  of  4223
3392  of  4223
3393  of  4223
3394  of  4223
3395  of  4223
3396  of  4223
3397  of  4223
3398  of  4223
3399  of  4223
3400  of  4223
3401  of  4223
3402  of  4223
3403  of  4223
3404  of  4223
3405  of  4223
3406  of  4223
3407  of  4223
3408  of  4223
3409  of  4223
3410  of  4223
3411  of  4223
3412  of  4223
3413  of  4223
3414  of  4223
3415  of  4223
3416  of  4223
3417  of  4223
3418  of  

3899  of  4223
3900  of  4223
3901  of  4223
3902  of  4223
3903  of  4223
3904  of  4223
3905  of  4223
3906  of  4223
3907  of  4223
3908  of  4223
3909  of  4223
3910  of  4223
3911  of  4223
3912  of  4223
3913  of  4223
3914  of  4223
3915  of  4223
3916  of  4223
3917  of  4223
3918  of  4223
3919  of  4223
3920  of  4223
3921  of  4223
3922  of  4223
3923  of  4223
3924  of  4223
3925  of  4223
3926  of  4223
3927  of  4223
3928  of  4223
3929  of  4223
3930  of  4223
3931  of  4223
3932  of  4223
3933  of  4223
3934  of  4223
3935  of  4223
3936  of  4223
3937  of  4223
3938  of  4223
3939  of  4223
3940  of  4223
3941  of  4223
3942  of  4223
3943  of  4223
3944  of  4223
3945  of  4223
3946  of  4223
3947  of  4223
3948  of  4223
3949  of  4223
3950  of  4223
3951  of  4223
3952  of  4223
3953  of  4223
3954  of  4223
3955  of  4223
3956  of  4223
3957  of  4223
3958  of  4223
3959  of  4223
3960  of  4223
3961  of  4223
3962  of  4223
3963  of  4223
3964  of  4223
3965  of  

In [96]:
# target_aperture = 7.838861042618269
# opt_singular_val_sum = 6.776236046088737
target_aperture = 7.206159960258321
opt_singular_val_sum = 6.586318544556513

### Kmeans helpers

In [97]:
# Distances


def compute_centroids(method, features_of_assignments):
    if method == "pred" or method == "sims" or method == "pred+centroids":
        centroids = []
        for feats in features_of_assignments:
            X = np.array([])
            for feat in feats:
                X = np.hstack((X, feat)) if X.size else feat
            if X != []:
                centroids.append( compute_c(X, aperture=target_aperture ) )
        #print("normalizing")
        centroids, _ = normalize_apertures(centroids, iterations=10, target_sum=opt_singular_val_sum)
        return centroids
        # for C_kmeans, C_kmeans_recomputed in zip(Cs_kmeans, Cs_kmeans_recomputed):
        #print("Mean divergence: ", d(C_kmeans, C_kmeans_recomputed)/np.linalg.norm(C_kmeans))
    else:
        return [ np.mean(feature,axis=0) for feature in features_of_assignments ]

def find_distances_to_centroids(method, p, feature, Cs, centroids, Ns=None, normalize=True):
    if method == "sims":
        sims = [ similarity_c(C,Cs[p]) for C in centroids ] # using the prev computed conceptors
        ds = [ 1-sim for sim in sims ]

    elif method == "pred":
        if Ns == None:
            Ns = Ns_from_Cs(centroids)
        es = evidences_for_Cs(feature,centroids,Ns)
        if XorZ == "X":
            es = [ np.sum(e) for e in es ]
        ds = es
    
    else:
        ds = [ d(feature,centroid) for centroid in centroids ]
    
    return [ d/sum(ds) for d in ds ] if normalize else ds

## Kmeans

In [99]:
def k_means(method, features, nb_clusters=4, max_epochs=15, new_assignments=None):

    if new_assignments == None:
        new_assignments = assign_to_clusters(n_samples, nb_clusters)
        #new_assignments = assign_to_clusters_smart(features, nb_clusters)

    for epoch in range(max_epochs):
        features_of_assignments = []
        for assignments in new_assignments:
            features_of_assignments.append([ features[a] for a in assignments ])

        centroids = compute_centroids(method, features_of_assignments)
        if method == "pred+centroids":
            centroids2 = compute_centroids("centroids", features_of_assignments)
        Ns = Ns_from_Cs(centroids) if method == "pred" or method == "pred+centroids" else None

        print("Epoch ",epoch,"# centroids:",len(centroids))

        old_assignments = new_assignments.copy()
        new_assignments = [ [] for _ in range(len(centroids)) ]

        for p, feature in enumerate(features):

            ### Find closest centroids

            if method == "pred+centroids":
                ds_pred = find_distances_to_centroids("pred", p, feature, Cs, centroids, Ns)
                ds_centroid = find_distances_to_centroids("centroids", p, feature, Cs, centroids2, Ns)
                ds = np.add(ds_pred, ds_centroid) / 2
            else:
                ds = find_distances_to_centroids(method, p, feature, Cs, centroids, Ns, normalize=False)
            centroid_index = np.argmin(ds)

            new_assignments[ centroid_index ].append(p)

        for new_assignment in new_assignments:
            stop = False
            for old_assignment in old_assignments:
                if set(new_assignment) == set(old_assignment):
                    stop = True
            if stop:
                print("Converged")
                return centroids, new_assignments
    return centroids, new_assignments


#Cs_kmeans, assignments_kmeans = k_means(method, kmeans_features,len(phonemes),100,smart_assignments)


## Single run

In [100]:
def single_run():
    method = "sims"
    if method == "pred" or method == "sims" or method=="centroids":
        kmeans_features = [ esn.run(feature.T, XorZ=XorZ) for feature in features ]
    else:
        kmeans_features = features


    save = False
    ass_name = "Smart_assignments"+file_name

    if save and os.path.exists('./cache/working/'+ass_name+'.pkl'):
        print("- from file")
        fp = open('./cache/working/'+ass_name+'.pkl','rb')
        smart_assignments = pkl.load(fp)
        fp.close()
        print("--- Done")
    else:
        print("- computing smart assignments")
        smart_assignments = assign_to_clusters_smart("sims", kmeans_features, Cs, len(phonemes))
        if save:
            fp = open("./cache/working/"+ass_name+".pkl",'wb')
            pkl.dump(smart_assignments,fp)
            fp.close()
        print("--- Done")


    plot = Plot(10,10)

    print("Kmeans NMI: ", NMI(assignments_kmeans, correct_assignments))
    print("Baseline NMI: ", NMI(assign_to_clusters(len(Cs),len(phonemes)), correct_assignments))
    print("Lengths correct: ", [len(x) for x in correct_assignments])
    print("Lengths kmeans: ", [len(x) for x in assignments_kmeans])

    plot.add_new_assignment_plot(correct_assignments,phonemes,xlabel="Sample index",ylabel="Original class membership")
    plot.add_new_assignment_plot(assignments_kmeans,smoothness=200,xlabel="Sample index",ylabel="Hard cluster membership \n(averaged over 10 adjacent samples)")
    plot.finalize()

## Experiment

In [101]:
print(len(features))

def experiment(iterations=5):
    params = {
        "method" : ["pred"],#["ogsignals", "centroids", "sims", "pred"],
        "smart" : [False]#[True, False]
    }
    results = []
    for method in params["method"]:
        for smart in params["smart"]:
            NMIs = []
            print("method", method, "smart", smart)
            if method == "pred" or method == "sims" or method == "pred+centroids" or method=="centroids":
                kmeans_features = [ esn.run(feature.T, XorZ=XorZ) for feature in features ]
            else:
                kmeans_features = features
            for i in range(iterations):
                if smart:
                    smart_assignments = assign_to_clusters_smart(method, kmeans_features, Cs, len(phonemes))
                else:
                    smart_assignments = None
                Cs_kmeans, assignments_kmeans = k_means(method,kmeans_features,len(phonemes),100,smart_assignments)
                NMIs.append(NMI(assignments_kmeans, correct_assignments))
            print("mean",np.mean(NMIs))
            results.append( {
                "method": method,
                "smart": smart,
                "NMI": np.mean(NMIs)
            } )
    return results

results = experiment(3)
print(results)

4223
method pred smart False
Target:  6.586318544556513
std 0.05698311988217593
Epoch  0 # centroids: 7
Target:  6.586318544556513
std 1.1865067052460925
Epoch  1 # centroids: 7
Target:  6.586318544556513
std 0.7750711195383068
Epoch  2 # centroids: 5
Target:  6.586318544556513
std 2.4179116987849865
Epoch  3 # centroids: 5
Target:  6.586318544556513
std 0.4155157633578382
Epoch  4 # centroids: 5
Target:  6.586318544556513
std 0.3920847960684362
Epoch  5 # centroids: 5
Target:  6.586318544556513
std 0.3699849970213977
Epoch  6 # centroids: 5
Target:  6.586318544556513
std 0.3737372749241185
Epoch  7 # centroids: 5
Target:  6.586318544556513
std 0.299330161297754
Epoch  8 # centroids: 5
Target:  6.586318544556513
std 0.3277640811934177
Epoch  9 # centroids: 5
Target:  6.586318544556513
std 0.32655308354393414
Epoch  10 # centroids: 5
Target:  6.586318544556513
std 0.31304953720144024
Epoch  11 # centroids: 5
Target:  6.586318544556513
std 0.3112885345638068
Epoch  12 # centroids: 5
Targ

Epoch  8 # centroids: 5
Target:  6.586318544556513
std 0.3277640811934177
Epoch  9 # centroids: 5
Target:  6.586318544556513
std 0.32655308354393414
Epoch  10 # centroids: 5
Target:  6.586318544556513
std 0.31304953720144024
Epoch  11 # centroids: 5
Target:  6.586318544556513
std 0.3112885345638068
Epoch  12 # centroids: 5
Target:  6.586318544556513
std 0.3285656756161993
Epoch  13 # centroids: 5
Target:  6.586318544556513
std 0.2906554778175866
Epoch  14 # centroids: 5
Target:  6.586318544556513
std 0.3363742839090951
Epoch  15 # centroids: 5
Target:  6.586318544556513
std 0.37684177383955336
Epoch  16 # centroids: 5
Target:  6.586318544556513
std 0.3794522649850406
Epoch  17 # centroids: 5
Target:  6.586318544556513
std 0.3224482397017585
Epoch  18 # centroids: 5
Target:  6.586318544556513
std 0.46128986909876313
Epoch  19 # centroids: 5
Target:  6.586318544556513
std 0.37295570864483385
Epoch  20 # centroids: 5
Target:  6.586318544556513
std 0.45924036074868163
Epoch  21 # centroids

Epoch  17 # centroids: 5
Target:  6.586318544556513
std 0.3224482397017585
Epoch  18 # centroids: 5
Target:  6.586318544556513
std 0.46128986909876313
Epoch  19 # centroids: 5
Target:  6.586318544556513
std 0.37295570864483385
Epoch  20 # centroids: 5
Target:  6.586318544556513
std 0.45924036074868163
Epoch  21 # centroids: 5
Target:  6.586318544556513
std 0.347631799933102
Epoch  22 # centroids: 5
Target:  6.586318544556513
std 0.44187227579201743
Epoch  23 # centroids: 5
Target:  6.586318544556513
std 0.4670106082106115
Epoch  24 # centroids: 5
Target:  6.586318544556513
std 0.42707812408746726
Epoch  25 # centroids: 5
Target:  6.586318544556513
std 0.4680687371150091
Epoch  26 # centroids: 5
Target:  6.586318544556513
std 0.42688908247422
Epoch  27 # centroids: 5
Target:  6.586318544556513
std 0.4652245206684984
Epoch  28 # centroids: 5
Target:  6.586318544556513
std 0.4254082827967339
Epoch  29 # centroids: 5
Target:  6.586318544556513
std 0.454813479090649
Epoch  30 # centroids: 5

In [57]:
print(len(features))

def experiment(iterations=5):
    params = {
        "method" : ["sims"],#["ogsignals", "centroids", "sims", "pred"],
        "smart" : [True]#[True, False]
    }
    results = []
    for method in params["method"]:
        for smart in params["smart"]:
            NMIs = []
            print("method", method, "smart", smart)
            if method == "pred" or method == "sims":
                kmeans_features = [ esn.run(feature.T, XorZ=XorZ) for feature in features ]
            else:
                kmeans_features = features
            for i in range(iterations):
                if smart:
                    smart_assignments = assign_to_clusters_smart(method, kmeans_features, Cs, len(phonemes))
                else:
                    smart_assignments = None
                Cs_kmeans, assignments_kmeans = k_means(method,kmeans_features,len(phonemes),100,smart_assignments)
                NMIs.append(NMI(assignments_kmeans, correct_assignments))
            print("mean",np.mean(NMIs))
            results.append( {
                "method": method,
                "smart": smart,
                "NMI": np.mean(NMIs)
            } )
    return results

results = experiment(1)
print(results)

427
method sims smart True
Finding centroid number  2


KeyboardInterrupt: 